# SUPG Advection Example: Dispersion and Phase Speed Diagrams

In [ ]:
# dependencies
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.develop(path="../..")
using Plots
using LFAToolkit
using LinearAlgebra 

In [ ]:
# parameters
p             = 2;
q             = p;
dimension     = 1;
mapping       = nothing;
collocate     = false;
c             = 1.0; # advective speed
τ             = 0.5 / (p - 1); # Tau scaling for SUPG, 0 returns Galerkin method
numbersteps   = 100;
maxeigenvalue = 0;
θ_min         = 0;
θ_max         = (p-1) * π;
θ_range       = LinRange(θ_min, θ_max, numbersteps);

In [ ]:
# setup
basis = TensorH1LagrangeBasis(p, q, 1, 1, collocatedquadrature = collocate, mapping = mapping)
Δx = 1.0

mesh = []
if dimension == 1
   mesh = Mesh1D(Δx)
elseif dimension == 2
   mesh = Mesh2D(Δx, Δx)
end

# Element mappings
dxdξ = Δx / 2 # 2 comes from quadrature domain of [-1,1]
dξdx = 1 / dxdξ
det_dxdξ = dxdξ # Determinant of mapping Jacobian

In [ ]:
# SUPG advection weak form 
function supgadvectionweakform(U::Matrix{Float64}, w::Array{Float64})
    u = U[1, :]
    du = U[2, :]
    dv = dξdx * (c * u - c * τ * (c * du) * dξdx) * w[1] * det_dxdξ
    return [dv]
end

# SUPG advection operator
inputs = [
    OperatorField(
        basis,
        [EvaluationMode.interpolation, EvaluationMode.gradient],
        "advected field",
    ),
    OperatorField(basis, [EvaluationMode.quadratureweights], "quadrature weights"),
]
outputs = [OperatorField(basis, [EvaluationMode.gradient])]
supgadvection = Operator(supgadvectionweakform, mesh, inputs, outputs)

# SUPG mass weak form and mass operator
function supgmassweakform(udot::Array{Float64}, w::Array{Float64})
    v = udot * w[1] * det_dxdξ
    dv = dξdx * c * τ * udot * w[1] * det_dxdξ
    return ([v; dv],)
end
supgmass = Operator(
    supgmassweakform,
    mesh,
    [
        OperatorField(basis, [EvaluationMode.interpolation], "u_t"),
        OperatorField(basis, [EvaluationMode.quadratureweights], "quadrature weights"),
    ],
    [OperatorField(basis, [EvaluationMode.interpolation, EvaluationMode.gradient])],
)

In [ ]:
# compute and plot dispersion and phase speed
# -- 1D --
function advectionsupgsymbol(θ_range)
    A = computesymbols(supgadvection, [θ_range]) # transform from reference to physical on dx=1 grid
    M = computesymbols(supgmass, [θ_range]) # mass matrix
    return sort(imag.(eigvals(-M \ A)))
end

eigenvalues = hcat(advectionsupgsymbol.(θ_range)...)

plot1 = plot(θ_range / π, eigenvalues' ./ π, linewidth = 3) # Dispersion diagram
        plot!(
        identity,
        xlabel = "θ/π",
        ylabel = "Eigenvalues",
        label = "exact",
        legend = :none,
        color = :black,
        title = "Dispersion relation, p=$p, collocate=$collocate, τ=$τ",
)
plot2 = plot(θ_range / π, eigenvalues' ./ θ_range, linewidth = 3) # Phase speed diagram
        plot!(
        one,
        xlabel = "θ/π",
        ylabel = "Eigvalues/θ",
        legend = :none,
        color = :black,
        title = "Phase speed, p=$p, collocate=$collocate, τ=$τ",
)
plot!(plot1, plot2, layout = (1, 2), size = (1400, 700))